# 02 - Customer Layer - ORIGINAL
 
Here the quantity and total spend per department will be added to the Customer Information (hh) table.

## Import Libraries

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import ast


import sys
#sys.path.append('/Users/antoniooliveira/Projects/NTT Project')
sys.path.append('C:/Users/aprataso/NTT_Project/Projects/NTT Project/')
import functions

## Import Data

In [4]:
#path = '/Users/antoniooliveira/Downloads/NTT project'
path = "C:/Users/aprataso/Downloads/final_data"

transaction = pd.read_csv(f"{path}/trans.csv")
prod = pd.read_csv(f"{path}/prod.csv")
hh = pd.read_csv(f"{path}/Gold/hh_treated.csv")

Merge the two datasets

In [5]:
merger = transaction.merge(prod, on = 'product_id', how = 'left')
merger.head(2)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,transaction_date,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,curr_size_of_product_value,curr_size_of_product_units
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,1.49,1.99,1.39,1,2021-01-01,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,5.0,LB
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.88,0.82,0.82,1,2021-01-01,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB,40.0,LB


In [6]:
merger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595732 entries, 0 to 2595731
Data columns (total 25 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   household_key               int64  
 1   basket_id                   int64  
 2   day                         int64  
 3   product_id                  int64  
 4   quantity                    int64  
 5   sales_value                 float64
 6   store_id                    int64  
 7   retail_disc                 float64
 8   trans_time                  int64  
 9   week_no                     int64  
 10  coupon_disc                 float64
 11  coupon_match_disc           float64
 12  sales_value_eu              float64
 13  loyalty_card_price          float64
 14  non_loyalty_card_price      float64
 15  days_                       int64  
 16  transaction_date            object 
 17  manufacturer                int64  
 18  department                  object 
 19  brand                

Since here we are using the non-Gold versions of the dataset, we have to reduce the number of departments again

In [7]:
print(merger['department'].unique())

replacement_mapping = {
    'MEAT': 'Meat',
    'MEAT-PCKGD': 'Meat',
    'MEAT-WHSE': 'Meat',
    'PORK': 'Meat',
    'SEAFOOD': 'Seafood',
    'SEAFOOD-PCKGD': 'Seafood',
    'FROZEN GROCERY': 'Groceries',
    'PRODUCE': 'Groceries',
    'GROCERY': 'Groceries',
    'NUTRITION': 'Groceries',
    'GRO BAKERY': 'Bakery',
    'PASTRY': 'Bakery',
    'DELI': 'Delicacies',
    'DAIRY DELI': 'Delicacies',
    'DELI/SNACK BAR': 'DELI/SNACK BAR',
    'PHOTO': 'Photo/Video',
    'VIDEO': 'Photo/Video'
}
merger['department'] = merger['department'].replace(replacement_mapping)


['PRODUCE' 'GROCERY' 'DRUG GM' 'MEAT' 'MEAT-PCKGD' 'DELI' 'SEAFOOD-PCKGD'
 ' ' 'PASTRY' 'NUTRITION' 'VIDEO RENTAL' 'MISC SALES TRAN' 'FLORAL'
 'SEAFOOD' 'SALAD BAR' 'AUTOMOTIVE' 'SPIRITS' 'COSMETICS' 'MISC. TRANS.'
 'GARDEN CENTER' 'CHEF SHOPPE' 'TRAVEL & LEISUR' 'COUP/STR & MFG'
 'KIOSK-GAS' 'FROZEN GROCERY' 'RESTAURANT' 'HOUSEWARES' 'PORK'
 'POSTAL CENTER' 'GM MERCH EXP' 'CNTRL/STORE SUP' 'PROD-WHS SALES'
 'DAIRY DELI' 'HBC' 'CHARITABLE CONT' 'RX' 'DELI/SNACK BAR'
 'PHARMACY SUPPLY' 'PHOTO' 'ELECT &PLUMBING' 'MEAT-WHSE' 'TOYS'
 'GRO BAKERY' 'VIDEO']


Verifying the success of the operation

In [8]:
merger.head(1)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,transaction_date,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,curr_size_of_product_value,curr_size_of_product_units
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,1.49,1.99,1.39,1,2021-01-01,69,Groceries,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,5.0,LB


Grouping our data by household_id to after join it with *hh*

In [9]:
agg_funcs = {
    'day': list, 
    #'basket_id': list,
    'product_id': list,  
    'quantity': lambda x: x.tolist(),   
    'sales_value': list,  
    'store_id': 'first',  
    'retail_disc':  'mean',  
    'trans_time': 'first', 
    'week_no': 'first',  
    'coupon_disc':  'mean',  
    'coupon_match_disc':  'mean',  
    'sales_value_eu':list,  
    'loyalty_card_price': list,  
    'non_loyalty_card_price': list, 
    'days_': 'first',  
    'transaction_date': 'first', 
    'manufacturer': list,  
    'department': list, 
    'brand': list,  
    'commodity_desc': list, 
    'sub_commodity_desc': list #,
    #'curr_size_of_product': list,  
    #'curr_size_of_product_value': list,  
    #'curr_size_of_product_units': list  
}

# Group by household_key and apply aggregations
grouped_df = merger.groupby(['household_key']).agg(agg_funcs).reset_index()

grouped_df['first_transaction_date'] = grouped_df['transaction_date']
grouped_df = grouped_df.drop('transaction_date', axis = 1)
grouped_df.head(2)

,household_key,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,manufacturer,department,brand,commodity_desc,sub_commodity_desc,first_transaction_date
0,1,"[51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 5...","[825123, 831447, 840361, 845307, 852014, 85498...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[3.99, 2.99, 1.09, 3.71, 2.79, 7.19, 2.5, 1.49...",436,-0.403613,1456,8,-0.046647,-0.015142,"[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[3.99, 2.99, 1.39, 4.33, 3.99, 8.49, 2.99, 1.4...","[3.99, 2.99, 1.09, 3.71, 2.79, 7.19, 2.5, 1.49...",51,"[1179, 317, 69, 4084, 69, 1011, 159, 407, 2082...","[Groceries, Groceries, Groceries, Delicacies, ...","[National, National, Private, National, Privat...","[SALD DRSNG/SNDWCH SPRD, CHEESE, EGGS, DELI ME...","[SEMI-SOLID SALAD DRESSING MAY, SHREDDED CHEES...",2021-02-20
1,2,"[103, 103, 103, 103, 103, 103, 103, 103, 103, ...","[854852, 930118, 1077555, 1098066, 5567388, 55...","[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, ...","[0.65, 4.47, 1.0, 0.79, 2.5, 2.5, 2.5, 5.0, 3....",401,-0.469174,1904,15,-0.012605,0.000000,"[0.7, 4.78, 1.07, 0.85, 2.68, 2.68, 2.68, 5.35...","[0.65, 9.5, 1.59, 0.99, 4.69, 4.69, 4.69, 4.69...","[0.65, 4.47, 1.0, 0.79, 2.5, 2.5, 2.5, 2.5, 3....",103,"[2, 2, 69, 69, 1208, 1208, 1208, 1208, 1323, 6...","[Groceries, Groceries, Groceries, Groceries, G...","[National, National, Private, Private, Nationa...","[TOMATOES, STONE FRUIT, MILK BY-PRODUCTS, BAKE...","[TOMATOES HOTHOUSE ON THE VINE, CHERRIES RED, ...",2021-04-13


Creating a column for the quantity spent in each department

In [10]:
'''unique_departments = set(dept for sublist in grouped_df['department'] for dept in sublist)

# Iterate over each row and compute department quantities
for dept in unique_departments:
    grouped_df[dept + '_quantity'] = grouped_df.apply(
        lambda row: sum(q for d, q in zip(row['department'], row['quantity']) if d == dept),
        axis=1
    )


grouped_df.head(2)'''

"unique_departments = set(dept for sublist in grouped_df['department'] for dept in sublist)\n\n# Iterate over each row and compute department quantities\nfor dept in unique_departments:\n    grouped_df[dept + '_quantity'] = grouped_df.apply(\n        lambda row: sum(q for d, q in zip(row['department'], row['quantity']) if d == dept),\n        axis=1\n    )\n\n\ngrouped_df.head(2)"

In [11]:
# grouped_df['quantity'].apply(functions.find_non_numeric_values)

Creating a dictionary that stores quantities bought per department per household

In [12]:
'''from collections import Counter

selected_household = grouped_df[grouped_df['household_key'] == 1]

# Extract the department lists for the selected household
departments_list = selected_household['department'].iloc[0]
quantities_list = selected_household['quantity'].iloc[0]

# Create a dictionary to store total quantities per department
department_quantities = {}
for dept, qty in zip(departments_list, quantities_list):
    if dept in department_quantities:
        department_quantities[dept] += qty
    else:
        department_quantities[dept] = qty

# Display the total quantity per department
print("Total quantity per department for household_key 1:")
for department, total_qty in department_quantities.items():
    print(f"{department}: {total_qty}")'''

'from collections import Counter\n\nselected_household = grouped_df[grouped_df[\'household_key\'] == 1]\n\n# Extract the department lists for the selected household\ndepartments_list = selected_household[\'department\'].iloc[0]\nquantities_list = selected_household[\'quantity\'].iloc[0]\n\n# Create a dictionary to store total quantities per department\ndepartment_quantities = {}\nfor dept, qty in zip(departments_list, quantities_list):\n    if dept in department_quantities:\n        department_quantities[dept] += qty\n    else:\n        department_quantities[dept] = qty\n\n# Display the total quantity per department\nprint("Total quantity per department for household_key 1:")\nfor department, total_qty in department_quantities.items():\n    print(f"{department}: {total_qty}")'

## Add information to Customer Info Dataset *hh*

In [13]:
hh.head(2)

,marital_status_code,homeowner_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s)
0,A,Homeowner,1,married,senior,2.0,0.0,65.0,42000,0.0,2,2.0
1,A,Homeowner,7,married,middle-aged,2.0,0.0,49.5,62000,0.0,2,2.0


DIFFERENT STEP HERE

In [14]:
clients = hh.merge(grouped_df, on='household_key', how='left')
clients

,marital_status_code,homeowner_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s),day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,manufacturer,department,brand,commodity_desc,sub_commodity_desc,first_transaction_date
0,A,Homeowner,1,married,senior,2.0,0.0,65.0,42000,0.0,2,2.0,"[51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 5...","[825123, 831447, 840361, 845307, 852014, 85498...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[3.99, 2.99, 1.09, 3.71, 2.79, 7.19, 2.5, 1.49...",436,-0.403613,1456,8,-0.046647,-0.015142,"[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[3.99, 2.99, 1.39, 4.33, 3.99, 8.49, 2.99, 1.4...","[3.99, 2.99, 1.09, 3.71, 2.79, 7.19, 2.5, 1.49...",51,"[1179, 317, 69, 4084, 69, 1011, 159, 407, 2082...","[Groceries, Groceries, Groceries, Delicacies, ...","[National, National, Private, National, Privat...","[SALD DRSNG/SNDWCH SPRD, CHEESE, EGGS, DELI ME...","[SEMI-SOLID SALAD DRESSING MAY, SHREDDED CHEES...",2021-02-20
1,A,Homeowner,7,married,middle-aged,2.0,0.0,49.5,62000,0.0,2,2.0,"[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 2...","[865569, 886703, 889731, 893400, 995436, 10205...","[2, 1, 3, 4, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, ...","[3.38, 1.09, 2.64, 4.0, 3.91, 3.86, 1.36, 2.29...",322,-0.375202,1435,4,-0.023639,-0.004005,"[3.62, 1.17, 2.82, 4.28, 4.18, 4.13, 1.46, 2.4...","[1.79, 1.09, 1.59, 1.39, 3.91, 5.79, 0.89, 2.4...","[1.69, 1.09, 0.88, 1.0, 3.91, 3.86, 0.68, 2.29...",23,"[418, 69, 69, 1251, 4680, 3050, 1251, 794, 69,...","[Groceries, Groceries, Groceries, Groceries, M...","[National, Private, Private, National, Nationa...","[CONVENIENT BRKFST/WHLSM SNACKS, MISC. DAIRY, ...","[GRANOLA BARS, REFRIGERATED MEXICAN PRODUCTS, ...",2021-01-23
2,U,Unknown,8,unknown,adult,2.0,1.0,29.5,29500,1.0,3,2.0,"[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 6...","[824550, 824555, 826578, 835576, 841220, 84798...","[1, 1, 1, 2, 3, 2, 1, 1, 1, 4, 1, 4, 1, 1, 2, ...","[3.49, 0.2, 3.99, 0.4, 11.37, 3.98, 0.88, 1.39...",321,-0.544871,2149,10,-0.014325,-0.000935,"[3.73, 0.21, 4.27, 0.43, 12.17, 4.26, 0.94, 1....","[3.49, 0.2, 3.99, 0.2, 3.79, 1.99, 0.99, 1.49,...","[3.49, 0.2, 3.99, 0.2, 3.79, 1.99, 0.88, 1.39,...",65,"[2264, 1046, 105, 1046, 2082, 1646, 69, 1768, ...","[Groceries, Groceries, DRUG GM, Groceries, Mea...","[National, National, National, National, Natio...","[MISC. DAIRY, PWDR/CRYSTL DRNK MX, DEODORANTS,...","[BREAD:TORTILLAS/PITA, SOFT DRINK POWDER POUCH...",2021-03-06
3,U,Homeowner,13,unknown,adult,2.0,1.0,29.5,87000,2.0,4,2.0,"[101, 101, 101, 101, 104, 104, 104, 104, 104, ...","[942385, 1106523, 5569230, 9883918, 840361, 84...","[1, 1, 5, 1, 1, 2, 1, 3, 1, 2, 5, 6, 2, 2, 1, ...","[12.74, 2.79, 12.5, 14.99, 1.09, 4.68, 0.86, 1...",323,-0.693011,1130,15,-0.131333,-0.033113,"[13.63, 2.99, 13.38, 16.04, 1.17, 5.01, 0.92, ...","[16.99, 2.79, 4.69, 19.99, 1.39, 2.69, 1.29, 0...","[12.74, 2.79, 2.5, 14.99, 1.09, 2.59, 0.86, 0....",101,"[6034, 69, 1208, 6034, 69, 531, 69, 693, 1136,...","[DRUG GM, Groceries, Groceries, DRUG GM, Groce...","[National, Private, National, National, Privat...","[SPRING/SUMMER SEASONAL, FLUID MILK PRODUCTS, ...","[INFLATABLES, FLUID MILK WHITE ONLY, SOFT DRIN...",2021-04-11
4,U,Homeowner,16,single,middle-aged,1.0,0.0,49.5,62000,0.0,1,0.0,"[98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 9...","[2378842, 2427281, 2625067, 7029671, 2403771, ...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, ...","[1.69, 2.89, 3.49, 2.39, 1.98, 0.79, 5.98, 4.9...",3316,-0.331315,657,15,-0.001934,0.000000,"[1.81, 3.09, 3.73, 2.56, 2.12, 0.85, 6.4, 5.33...","[1.69, 2.89, 3.49, 2.39, 2.33, 0.79, 2.99, 2.4...","[1.69, 2.89, 3.49, 2.39, 1.98, 0.79, 2.99, 2.4...",98,"[1804, 194, 1089, 136, 3177, 69, 75, 194, 4841...","[Groceries, Groceries, Meat, Groceries, Grocer...","[National, National, National, National, Natio...","[

Joining *merge* with *hh*, which means we will keep 2500 rows, even tho 1699 will have missing values as we do not have information about those customers yet

In [15]:
'''clients = grouped_df.merge(hh, on='household_key', how='left')
clients'''

"clients = grouped_df.merge(hh, on='household_key', how='left')\nclients"

Dropping columns that will not be necessary

In [16]:
columns_to_drop = [
    'day', 'product_id', 'sales_value',
    'store_id','trans_time', 'week_no',
    'loyalty_card_price',
    'non_loyalty_card_price', 'days_', 'manufacturer',
    'brand', 'commodity_desc', 'sub_commodity_desc'
]


clients_cleaned = clients.drop(columns=columns_to_drop)
clients_cleaned.head(2)

,marital_status_code,homeowner_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s),quantity,retail_disc,coupon_disc,coupon_match_disc,sales_value_eu,department,first_transaction_date
0,A,Homeowner,1,married,senior,2.0,0.0,65.0,42000,0.0,2,2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",-0.403613,-0.046647,-0.015142,"[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[Groceries, Groceries, Groceries, Delicacies, ...",2021-02-20
1,A,Homeowner,7,married,middle-aged,2.0,0.0,49.5,62000,0.0,2,2.0,"[2, 1, 3, 4, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, ...",-0.375202,-0.023639,-0.004005,"[3.62, 1.17, 2.82, 4.28, 4.18, 4.13, 1.46, 2.4...","[Groceries, Groceries, Groceries, Groceries, M...",2021-01-23


Create a column for each department that represents the amount of money spent per department per household

In [17]:
# Convert string representations of lists to actual lists if needed
clients_cleaned['department_'] = clients_cleaned['department'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
clients_cleaned['sales_value_eu_'] = clients_cleaned['sales_value_eu'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
clients_cleaned['quantity_'] = clients_cleaned['quantity'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convert lists within DataFrame columns to numeric values if necessary
clients_cleaned['sales_value_eu_'] = clients_cleaned['sales_value_eu'].apply(lambda x: list(map(float, x)))
clients_cleaned['quantit_y'] = clients_cleaned['quantity'].apply(lambda x: list(map(float, x)))

# Initialize an empty list to collect rows
rows = []

# Iterate over each row in the DataFrame
for index, row in clients_cleaned.iterrows():
    household_key = row['household_key']
    departments = row['department']  # Ensure this is a list of department names
    sales_values = row['sales_value_eu']  # Ensure this is a list of sales values (numeric)
    quantities = row['quantity']  # Ensure this is a list of quantities (numeric)

    # Create a dictionary to store the total spending for each department
    total_spending = {}
    
    for dept, sales_value, quantity in zip(departments, sales_values, quantities):
        if dept not in total_spending:
            total_spending[dept] = 0
        total_spending[dept] += sales_value
    
    # Convert the dictionary to a DataFrame row
    spending_row = {'household_key': household_key}
    spending_row.update(total_spending)
    
    # Append the row to the list
    rows.append(spending_row)

# Convert the list of rows into a DataFrame
spending_per_department = pd.DataFrame(rows)

# Fill NaN values with 0 (for departments with no spending)
spending_per_department.fillna(0, inplace=True)


# Join the spending DataFrame to clients_cleaned
clients_cleaned = clients_cleaned.merge(spending_per_department, on='household_key', how='left')

# Rename spending columns to include '_spend'
spending_columns = spending_per_department.columns.drop('household_key')
spending_column_mapping = {col: f"{col}_spend" for col in spending_columns}

# Rename the columns
clients_cleaned.rename(columns=spending_column_mapping, inplace=True)

# Display the result
clients_cleaned.head(2)


,marital_status_code,homeowner_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s),quantity,retail_disc,coupon_disc,coupon_match_disc,sales_value_eu,department,first_transaction_date,department_,sales_value_eu_,quantity_,quantit_y,Groceries_spend,Delicacies_spend,Meat_spend,Bakery_spend,DRUG GM_spend,_spend,SALAD BAR_spend,MISC SALES TRAN_spend,RESTAURANT_spend,FLORAL_spend,COSMETICS_spend,MISC. TRANS._spend,Seafood_spend,KIOSK-GAS_spend,SPIRITS_spend,CHEF SHOPPE_spend,GARDEN CENTER_spend,AUTOMOTIVE_spend,CNTRL/STORE SUP_spend,COUP/STR & MFG_spend,TRAVEL & LEISUR_spend,GM MERCH EXP_spend,POSTAL CENTER_spend,DELI/SNACK BAR_spend,Photo/Video_spend,RX_spend,PROD-WHS SALES_spend,PHARMACY SUPPLY_spend,TOYS_spend,HOUSEWARES_spend,HBC_spend,VIDEO RENTAL_spend
0,A,Homeowner,1,married,senior,2.0,0.0,65.0,42000,0.0,2,2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",-0.403613,-0.046647,-0.015142,"[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[Groceries, Groceries, Groceries, Delicacies, ...",2021-02-20,"[Groceries, Groceries, Groceries, Delicacies, ...","[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, ...",3285.02,226.99,373.56,102.01,568.23,0.0,44.09,21.4,4.47,8.55,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,Homeowner,7,married,middle-aged,2.0,0.0,49.5,62000,0.0,2,2.0,"[2, 1, 3, 4, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, ...",-0.375202,-0.023639,-0.004005,"[3.62, 1.17, 2.82, 4.28, 4.18, 4.13, 1.46, 2.4...","[Groceries, Groceries, Groceries, Groceries, M...",2021-01-23,"[Groceries, Groceries, Groceries, Groceries, M...","[3.62, 1.17, 2.82, 4.28, 4.18, 4.13, 1.46, 2.4...","[2, 1, 3, 4, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, ...","[2.0, 1.0, 3.0, 4.0, 1.0, 1.0, 2.0, 1.0, 1.0, ...",2342.43,166.01,288.04,27.62,698.65,0.0,0.00,0.0,0.00,5.34,101.34,1.87,7.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


As the *quantity* columns will not be needed, they will be dropped

In [18]:
quantity_columns = [col for col in clients_cleaned.columns if '_quantity' in col]

clients_cleaned.drop(columns=quantity_columns, inplace=True)

Drop temporary variables

In [19]:
clients_cleaned = clients_cleaned.drop([' _spend', 'quantit_y', 'quantity_', 'sales_value_eu_', 'department_'], axis = 1) #'dep_quantity'

Serialize variables to keep their data type after export

In [20]:
# Apply serialization
clients_cleaned['quantity'] = clients_cleaned['quantity'].apply(functions.serialize_list)
clients_cleaned['sales_value_eu'] = clients_cleaned['sales_value_eu'].apply(functions.serialize_list)
clients_cleaned['department'] = clients_cleaned['department'].apply(functions.serialize_list)

In [21]:
clients_cleaned

,marital_status_code,homeowner_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s),quantity,retail_disc,coupon_disc,coupon_match_disc,sales_value_eu,department,first_transaction_date,Groceries_spend,Delicacies_spend,Meat_spend,Bakery_spend,DRUG GM_spend,SALAD BAR_spend,MISC SALES TRAN_spend,RESTAURANT_spend,FLORAL_spend,COSMETICS_spend,MISC. TRANS._spend,Seafood_spend,KIOSK-GAS_spend,SPIRITS_spend,CHEF SHOPPE_spend,GARDEN CENTER_spend,AUTOMOTIVE_spend,CNTRL/STORE SUP_spend,COUP/STR & MFG_spend,TRAVEL & LEISUR_spend,GM MERCH EXP_spend,POSTAL CENTER_spend,DELI/SNACK BAR_spend,Photo/Video_spend,RX_spend,PROD-WHS SALES_spend,PHARMACY SUPPLY_spend,TOYS_spend,HOUSEWARES_spend,HBC_spend,VIDEO RENTAL_spend
0,A,Homeowner,1,married,senior,2.0,0.0,65.0,42000,0.0,2,2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",-0.403613,-0.046647,-0.015142,"[4.27, 3.2, 1.17, 3.97, 2.99, 7.69, 2.68, 1.59...","[""Groceries"", ""Groceries"", ""Groceries"", ""Delic...",2021-02-20,3285.02,226.99,373.56,102.01,568.23,44.09,21.40,4.47,8.55,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,Homeowner,7,married,middle-aged,2.0,0.0,49.5,62000,0.0,2,2.0,"[2, 1, 3, 4, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, ...",-0.375202,-0.023639,-0.004005,"[3.62, 1.17, 2.82, 4.28, 4.18, 4.13, 1.46, 2.4...","[""Groceries"", ""Groceries"", ""Groceries"", ""Groce...",2021-01-23,2342.43,166.01,288.04,27.62,698.65,0.00,0.00,0.00,5.34,101.34,1.87,7.05,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,U,Unknown,8,unknown,adult,2.0,1.0,29.5,29500,1.0,3,2.0,"[1, 1, 1, 2, 3, 2, 1, 1, 1, 4, 1, 4, 1, 1, 2, ...",-0.544871,-0.014325,-0.000935,"[3.73, 0.21, 4.27, 0.43, 12.17, 4.26, 0.94, 1....","[""Groceries"", ""Groceries"", ""DRUG GM"", ""Groceri...",2021-03-06,3699.90,153.43,626.66,49.47,833.41,70.22,1.69,0.00,142.23,28.24,0.00,42.80,202.17,53.45,5.36,14.27,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,U,Homeowner,13,unknown,adult,2.0,1.0,29.5,87000,2.0,4,2.0,"[1, 1, 5, 1, 1, 2, 1, 3, 1, 2, 5, 6, 2, 2, 1, ...",-0.693011,-0.131333,-0.033113,"[13.63, 2.99, 13.38, 16.04, 1.17, 5.01, 0.92, ...","[""DRUG GM"", ""Groceries"", ""Groceries"", ""DRUG GM...",2021-04-11,5602.83,104.85,381.78,9.10,3077.93,10.80,1370.50,0.00,50.24,44.34,0.00,25.53,3429.51,0.00,0.00,0.00,7.69,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,U,Homeowner,16,single,middle-aged,1.0,0.0,49.5,62000,0.0,1,0.0,"[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, ...",-0.331315,-0.001934,0.000000,"[1.81, 3.09, 3.73, 2.56, 2.12, 0.85, 6.4, 5.33...","[""Groceries"", ""Groceries"", ""Meat"", ""Groceries""...",2021-04-08,984.68,104.74,161.16,81.83,73.08,6.88,0.00,0.00,0.00,0.00,0.00,0.00,205.76,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,U,Homeowner,2494,unknown,middle-aged,2.0,0.0,39.5,62000,0.0,2,2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...",-0.716337,-0.031426,-0.001459,"[1.7, 0.95, 1.06, 7.48, 0.74, 8.34, 5.19, 3.73...","[""Groceries"", ""Groceries"", ""Groceries"", ""Groce...",2021-04-24,1443.81,95.71,200.78,63.78,277.64,0.00,0.00,0.00,8.54,8.17,0.00,29.43,120.85,7.48,0.00,0.00,13.43,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,A,Homeowner,2496,married,middle-aged,2.0,0.0,49.5,87000,0.0,3,2.0,"[1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 6, 1, 1, 1, ...",-0.748885,-0.034654,-0.008328,"[0.85, 3.97, 3.16, 1.17, 3.2, 5.69, 17.38, 1.8...","[""Groceries"", ""Meat"", ""Groceries"", ""Groceries""...",2021-04-27,2255.22,105.82,1383.51,31.68,357.55,0.00,68.56,0.00,0.00,0.00,0.00,80.77,346.69,12.83,0.00,0.00,0.00,0.0,0.0,0.00,0.0,1.69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
798,U,Unknown,2497,single,middle-aged,1

## Export

In [22]:
clients_cleaned.to_csv(f'{path}/original_data/801_cust_info.csv', index = True)